# Éditeur EXIF avec Streamlit

Ce notebook permet de visualiser et de modifier les métadonnées EXIF d’une image à l’aide de Streamlit.

In [ ]:
import streamlit as st
from PIL import Image
from PIL.ExifTags import TAGS
import piexif

In [ ]:
# Widget Streamlit pour charger une image (jpg, jpeg, png)
uploaded_file = st.file_uploader("Chargez une photo", type=["jpg", "jpeg", "png"])

if uploaded_file:
    # Ouvre l'image avec Pillow
    image = Image.open(uploaded_file)
    # Affiche l'image dans l'application Streamlit avec une légende
    st.image(image, caption="Votre photo", use_column_width=True)

In [ ]:
def get_exif_data(img):
    """
    Extrait les métadonnées EXIF d'une image PIL et les retourne sous forme de dictionnaire lisible.
    """
    exif_data = {}
    info = img._getexif()
    if info:
        for tag, value in info.items():
            decoded = TAGS.get(tag, tag)
            exif_data[decoded] = value
    return exif_data

In [ ]:
if uploaded_file:
    # Extraction et affichage des métadonnées EXIF existantes
    exif_data = get_exif_data(image)
    st.write("Métadonnées EXIF existantes :", exif_data)

In [ ]:
if uploaded_file:
    # Champ de saisie pour modifier le champ 'Artist' (auteur) dans les EXIF
    artist = st.text_input("Auteur/Artiste", value=exif_data.get('Artist', ''))

    if st.button("Mettre à jour les EXIF"):
        # Charge les EXIF existantes avec piexif (nécessite que l'image ait des EXIF)
        exif_bytes = image.info.get("exif", None)
        if exif_bytes:
            exif_dict = piexif.load(exif_bytes)
            exif_dict['0th'][piexif.ImageIFD.Artist] = artist.encode('utf-8')
            new_exif = piexif.dump(exif_dict)
            image.save("output.jpg", exif=new_exif)
            st.success("EXIF mis à jour et image sauvegardée.")
        else:
            st.error("Aucune donnée EXIF trouvée dans l'image.")